`AutoBuild`, a pipeline that can **automatically build multi-agent systems** for complex tasks. 

`AutoBuild` is an effective approach that generates a group of experts and use their conversation to solve a task. In `AutoBuild`, each expert handles a part of the task, therefore effectively and comprehensively solving it.

`AgentBuilder`, which will complete the generation of participant expert agents and the construction of group chat automatically after the user provides descriptions of a building task and an execution task

In [1]:
AUTOBUILD_SYSTEM_MESSAGE = """You are a manager of a group of advanced experts, your primary objective is to delegate the resolution of tasks to other experts through structured dialogue and derive conclusive insights from their conversation summarization.
When a task is assigned, it's crucial to assess its constraints and conditions for completion. If feasible, the task should be divided into smaller, logically consistent subtasks. Following this division, you have the option to address these subtasks by forming a team of agents using the "autobuild" tool.
Upon the completion of all tasks and verifications, you should conclude the operation and reply "TERMINATE".
"""

In [ ]:
import os
from datetime import datetime
from typing import Callable, Dict, Literal, Optional, Union

from typing_extensions import Annotated
# Install the autogen package
import autogen
from autogen import (
    Agent,
    AssistantAgent,
    ConversableAgent,
    GroupChat,
    GroupChatManager,
    UserProxyAgent,
    register_function,
)
from autogen.agentchat.contrib import agent_builder
from autogen.cache import Cache
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor


In [3]:
config_list = autogen.config_list_from_json(
    env_or_file="AOAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": {
            "gpt-4o"
        }
    },
)

In [4]:
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
)

In [5]:
autobuild_assistant = AssistantAgent(
    name="Autobuild Assistant",
    llm_config={"config_list": config_list, "cache_seed": None},
)

In [6]:
def autobuild_reply(recipient, messages, sender, config):
    last_msg = messages[-1]["content"]
    builder = agent_builder.AgentBuilder(
        config_file_or_env="AOAI_CONFIG_LIST",
        builder_model="gpt-4o",
        agent_model="gpt-4o",
    )
    agent_list, agent_configs = builder.build(
        last_msg, 
        default_llm_config={"config_list": config_list, "cache_seed": None},
        coding=True,
        code_execution_config= {
            "work_dir": "coding",
            "use_docker": False
        }
    )
    # start nested chat
    nested_group_chat = GroupChat(
        agents=agent_list,
        messages=[],
    )
    manager = GroupChatManager(groupchat=nested_group_chat, llm_config={"config_list": config_list, "cache_seed": None})
    chat_res = agent_list[0].initiate_chat(
        manager, message=agent_configs.get("building_task", last_msg), summary_method="reflection_with_llm"
    )
    return True, chat_res.summary

In [7]:
autobuild_assistant.register_reply([Agent, None], autobuild_reply)

In [8]:
task = (
    f"Today is {datetime.now().date()}. Write a blogpost about the stock price performance of Nvidia in the past month."
)

In [ ]:
with Cache.disk(cache_seed=41) as cache:
    user_proxy.initiate_chat(autobuild_assistant, message=task, max_turns=1)